## Импорты

In [2]:
import pandas as pd
import numpy as np
from sklearn.utils import check_array

import warnings
warnings.filterwarnings('ignore')

from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split

from keras.layers import Dense,Dropout,BatchNormalization ,Input,Conv1D
from keras.models import Model
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.callbacks import LearningRateScheduler,TensorBoard



PATH_TO_DATA="/home/roman/finunichamp2018/1/"
PATH_TO_SUBMIT="/home/roman/finunichamp2018/1/answers_archive/"

In [3]:
def submit(pred,name):
    sub=pd.read_csv(PATH_TO_DATA+"insvalue_sample.csv",index_col="id")
    sub["target"]=pred
    sub.to_csv(PATH_TO_SUBMIT+name+".csv")

In [4]:
def lr_decay_callback(lr_init, lr_decay):
    def step_decay(epoch):
        return lr_init * (lr_decay ** (epoch + 1))
    return LearningRateScheduler(step_decay)

In [5]:
X_train_lin=pd.read_csv(PATH_TO_DATA+"X_train_lin.csv")
X_test_lin=pd.read_csv(PATH_TO_DATA+"X_test_lin.csv")

y=pd.read_csv(PATH_TO_DATA+"Y.csv")
y=y["target"]

np.random.seed(17)

In [6]:
data=X_train_lin

batch_size=8192
epochs=2000

X_train, X_test, y_train, y_test=train_test_split(X_train_lin,y,test_size=0.3,random_state=17)

In [7]:
#try10
inp=Input((data.shape[1],))

dense1=Dense(128, activation='selu',kernel_initializer='he_uniform')(inp)
batch_norm1=BatchNormalization()(dense1)
dropout1=Dropout(0.2)(batch_norm1)

dense2=Dense(256, activation='selu',kernel_initializer='he_uniform')(dropout1)
batch_norm2=BatchNormalization()(dense2)
dropout2=Dropout(0.3)(batch_norm2)

dense3=Dense(512, activation='selu',kernel_initializer='he_uniform')(dropout2)
batch_norm3=BatchNormalization()(dense3)
dropout3=Dropout(0.3)(batch_norm3)

dense4=Dense(1024, activation='selu',kernel_initializer='he_uniform')(dropout3)
batch_norm4=BatchNormalization()(dense4)
dropout5=Dropout(0.3)(batch_norm4)

dense5=Dense(128, activation='selu',kernel_initializer='he_uniform')(dropout5)

out=Dense(1)(dense5)

model=Model(inputs=inp,outputs=out)
model.compile(optimizer="Nadam",
              loss="mean_absolute_percentage_error",
              metrics=["mean_absolute_percentage_error"])

In [ ]:
model.fit(X_train_lin,
          y,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.1,
          verbose=2)

In [ ]:
# history
# 9.8819
# 9.2800
# 9.1571

In [11]:
pred=model.predict(X_test_lin)

In [12]:
submit(pred,"nn_arh10")